# Average patterns 
With this notebook we want to see the average pattern over different timescales

In [1]:
# Import the necessary external libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
# Import the necessary internal functions
import sys
sys.path.append('..')
from eval.evaluator import load_data
from preprocessing.advanced_transforms import load_characteristics
dmas_h_q, wea_h = load_data()
dmas_characteristics = load_characteristics()
DAY_LEN = 24
WEEK_LEN = 7 * DAY_LEN
WGROUP_LEN = 4 * WEEK_LEN # If we group regularly with grous of 4 weeks we have 13 months
YEAR_LEN = 52 * WEEK_LEN

## Daily patterns 

### Scaled average daily patterns 
This fig shows the behaviour of the consumption Q over a day for each DMA and the whole network.

In [3]:
# Plot the average consumption of a day at the hourly level
dmas_h_q['Network'] = dmas_h_q.sum(axis=1, skipna=False)
colors = px.colors.qualitative.Plotly.copy()
colors.append('#000000')

dmas_h_q__byhour = dmas_h_q.groupby(dmas_h_q.index.hour)

fig = go.Figure()

for i, dma in enumerate(dmas_h_q.columns):
    fig.add_trace(go.Scatter(x=dmas_h_q__byhour[dma].mean().index, 
                             y=dmas_h_q__byhour[dma].mean()/dmas_h_q__byhour[dma].mean().mean(), 
                             name=dma,
                             text=dmas_h_q__byhour[dma].mean(),
                             line=dict(color=colors[i]),
                             hovertemplate='Hour %{x}: %{text:.1f} L/s <b>'+dma+'</b>'
                            )
                    )

fig.update_layout(
    title='Average consumption by Hour',
    xaxis_title='Hour of the day',
    yaxis_title=r'$Q_d / \bar{Q}_d$',
    legend_title='DMA',
    xaxis=dict(
        range=[-0.2, 23.2],
        tickmode='array',
        tickvals=list(range(0, 25, 2)),
        ticktext=list(range(0, 25, 2))
    )
)

fig.show()

In [4]:
# 
fig = go.Figure()

for i, dma in enumerate(dmas_h_q.columns):
    fig.add_trace(go.Scatterpolar(r=dmas_h_q__byhour[dma].mean()/dmas_h_q__byhour[dma].mean().mean(),
                                      theta=dmas_h_q__byhour[dma].mean().index/DAY_LEN*360,
                                      name=dma,
                                      text=dmas_h_q__byhour[dma].mean(),
                                      line=dict(color=colors[i]),
                                      hovertemplate='Hour %{theta}: %{text:.1f} L/s <b>'+dma+'</b>'
                                     )
                     )


fig.update_layout(
    title='Average consumption by Hour',
    legend_title='DMA',
    polar=dict(
        radialaxis=dict(
            range=[0, 1.5],
            tickmode='array',
            tickvals=[0, 0.5, 1, 1.5],
            ticktext=[0, 0.5, 1, 1.5],
            tickangle=0
        ),
        angularaxis=dict(
            rotation=0,
            tickmode='array',
            tickvals=list(range(0, 360, 30)),
            ticktext=list(range(0, 24, 2)),
            direction='clockwise'
        )
    )
)

fig.show()

### Holiday vs weekday

In [5]:
# Plot the average consumption of a weekday and non-weekday at the hourly level
from Utils.data_loader import load_calendar
calendar = load_calendar()
calendar['Holiday'] = calendar['Holiday'].astype(bool)
calendar['Weekend'] = calendar['Weekend'].astype(bool)
calendar['NonWorkDay'] = calendar['Holiday'] | calendar['Weekend']
calendar = calendar.resample('H').ffill()
calendar = calendar.loc[calendar.index.isin(dmas_h_q.index)]

fig = go.Figure()

for i, dma in enumerate(dmas_h_q.columns):
    dma_h_q_workday = dmas_h_q.loc[~calendar['NonWorkDay'],dma]
    dma_h_q_workday__byh = dma_h_q_workday.groupby(dma_h_q_workday.index.hour)
    fig.add_trace(go.Scatter(x=dma_h_q_workday__byh.mean().index, 
                             y=dma_h_q_workday__byh.mean()/dmas_h_q[dma].mean(), 
                             text=dma_h_q_workday__byh.mean(),
                             name="Workday",
                             legendgroup=dma,
                             legendgrouptitle=dict(text=dma),
                             line=dict(color=colors[i]),
                             hovertemplate="(%{x}): %{text:.1f} L/s <b>"+dma+"</b>")
                )
    
    dma_h_q_Nwd = dmas_h_q.loc[calendar['NonWorkDay'],dma]
    dma_h_q_Nwd__byh = dma_h_q_Nwd.groupby(dma_h_q_Nwd.index.hour)
    fig.add_trace(go.Scatter(x=dma_h_q_Nwd__byh.mean().index, 
                             y=dma_h_q_Nwd__byh.mean()/dmas_h_q[dma].mean(),
                             text=dma_h_q_Nwd__byh.mean(),
                             name="Non wd",
                             legendgroup=dma,
                             line=dict(color=colors[i],
                                       dash='dash'),
                             hovertemplate="(%{x}): %{text:.1f} L/s <b>"+dma+"</b>"),
                )
    
fig.update_layout(
    title='Average by Hour (Workdays vs Non-Workdays)',
    xaxis=dict(
        range=[-0.2, 23.2],
        tickmode='array',
        tickvals=list(range(0, 25, 2)),
        ticktext=None
    ),
    yaxis=dict(
        title='Average',
    ),
    legend=dict(title='DMA',
                groupclick='toggleitem')
)

fig.show()

In [6]:
# Same but in polars

## Average daily pattern of each day of the week

In [7]:
# Plot the average pattern of the consumption every day of the week
fig = go.Figure()

dmas_h_q__byday = dmas_h_q.groupby(dmas_h_q.index.dayofweek)

for i, dma in enumerate(dmas_h_q.columns):
    dma_h_q__week = np.zeros(WEEK_LEN)

    for d in range(0,7):
        dma_h_q__day = dmas_h_q__byday.get_group(d)[dma]
        dma_h_q__day__byh = dma_h_q__day.groupby(dma_h_q__day.index.hour)

        dma_h_q__week[d*DAY_LEN:(d+1)*DAY_LEN] = dma_h_q__day__byh.mean().values

    fig.add_trace(go.Scatter(x=np.arange(0, WEEK_LEN),
                            y=dma_h_q__week/dma_h_q__week.mean(),
                            name=dma,
                            text=dma_h_q__week,
                            line=dict(color=colors[i]),
                            hovertemplate="(%{x}): %{text:.1f} L/s <b>"+dma+"</b>")
                    )
    

fig.update_layout(
    title='Average by Hour (Weekdays)',
    xaxis=dict(
        range=[-0.2, WEEK_LEN+0.2],
        tickmode='array',
        tickvals=list(range(0, WEEK_LEN+1, DAY_LEN)),
        ticktext=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon']
    ),
    yaxis=dict(
        title='Average',
    ),
    legend=dict(title='DMA')
)


## Average week pattern for every quarter of the year 

In [8]:
# Similar as above

quarters = ['Q1-Wi', 'Q2-Sp', 'Q3-Su', 'Q4-Au']
quarters_months = [[12,1,2], [3,4,5], [6,7,8], [9,10,11]]
styles = ['solid', 'dash', 'dot', 'dashdot']

fig = go.Figure()

for i, dma in enumerate(dmas_h_q.columns):
    for q in range(0, 4):
        dma_h_q__quarter = dmas_h_q[dmas_h_q.index.month.isin(quarters_months[q])][dma]
        
        dma_h_q__quarter__byday = dma_h_q__quarter.groupby(dma_h_q__quarter.index.dayofweek)

        dma_h_q__quarter__week = np.zeros(WEEK_LEN)
        for d in range(0,7):
            dma_h_q__quarter__day = dma_h_q__quarter__byday.get_group(d)
            dma_h_q__quarter__day = dma_h_q__quarter__day.groupby(dma_h_q__quarter__day.index.hour).mean()
            
            dma_h_q__quarter__week[d*DAY_LEN:(d+1)*DAY_LEN] = dma_h_q__quarter__day.values

        fig.add_trace(go.Scatter(x=np.arange(0, WEEK_LEN),
                                y=dma_h_q__quarter__week/dmas_h_q[dma].mean(),
                                name=quarters[q],
                                legendgroup=dma,
                                legendgrouptitle=dict(text=dma),
                                text=dma_h_q__quarter__week,
                                line=dict(color=colors[i],
                                          dash=styles[q]),
                                hovertemplate="(%{x}): %{text:.1f} L/s <b>"+dma+"</b>")
                        )
    

fig.update_layout(
    title='Average by Hour (Quarters)',
    xaxis=dict(
        range=[-0.2, WEEK_LEN+0.2],
        tickmode='array',
        tickvals=list(range(0, WEEK_LEN+1, DAY_LEN)),
        ticktext=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon']
    ),
    yaxis=dict(
        title='Average',
    ),
    legend=dict(title='DMA')
)
fig.show()




